<a href="https://colab.research.google.com/github/wlgns410/pandas/blob/main/3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
cd drive/MyDrive/drive/MyDrive/RecoSys/Data

/content/drive/MyDrive/drive/MyDrive/RecoSys/Data


In [6]:
base_src = '/content/drive/MyDrive/drive/MyDrive/RecoSys/Data'

u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src, sep='|', names=u_cols, encoding='latin-1')

u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release date', 'video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
          'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src, sep='|', names=i_cols, encoding='latin-1')

u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t', names=r_cols, encoding='latin-1')

# ratings DataFrame에서 timestamp 제거
ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

x = ratings.copy()
y = ratings['user_id']

### test_size: 25%, train:75%로 구성하겠다는 뜻  stratify는 데이터 뭉침을 방지 위함
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, stratify=y)

# 정확도(RMSE) 계산 함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) **2))

# 모델별 RMSE 계산하는 함수
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user,movie,neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y)
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

### 코사인 유사도 계산###
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)


In [9]:
## 주어진 영화의 가중평균을 계산하는 함수
def CF_simple(user_id, movie_id, neighbor_size=0):
  if movie_id in rating_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores,movie_ratings) / sim_scores.sum()

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0
  return mean_rating


## 정확도 계산
score(CF_simple, neighbor_size=30)





0.9033178936211771

In [14]:
## 실제 주어진 사용자에 대해 추천을 받는 기능
rating_matrix = ratings.pivot_table(values='rating',
                                    index='user_id',
                                    columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

def recom_movie(user_id, n_items, neighbor_size=30):
  user_movie = rating_matrix.loc[user_id].copy()

  for movie in rating_matrix.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0

    else:
      user_movie.loc[movie] = CF_simple(user_id, movie,neighbor_size)
  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(user_id=729,n_items=5,neighbor_size=30)

movie_id
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
1467                                     Cure, The (1995)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
22                                     Taxi Driver (1976)
Name: title, dtype: object